In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from datetime import datetime,date,timedelta
import pickle

In [4]:
prevwind = pd.read_pickle('brommer_raw_2020-10-04_2022-10-03.pkl')
prevwind['Month'] = pd.DatetimeIndex(prevwind['Date']).month_name().str[:3]

In [5]:
prevwind

,Date,Time,Temp,Wind,Orient,Speed,Gust,Month
0,2020-10-04,12:04 AM,55.6,East,90.0,0.0,0.0,Oct
1,2020-10-04,12:09 AM,55.4,East,90.0,0.0,0.0,Oct
2,2020-10-04,12:14 AM,55.3,East,90.0,0.0,0.0,Oct
3,2020-10-04,12:19 AM,55.2,East,90.0,0.0,0.0,Oct
4,2020-10-04,12:24 AM,55.2,East,90.0,0.0,0.0,Oct
...,...,...,...,...,...,...,...,...
283,2022-10-03,11:39 PM,57.8,WNW,292.5,0.2,2.0,Oct
284,2022-10-03,11:44 PM,57.6,West,270.0,0.2,2.0,Oct
285,2022-10-03,11:49 PM,57.6,West,270.0,0.0,2.0,Oct
286,2022-10-03,11:54 PM,57.4,West,270.0,0.0,2.0,Oct


In [ ]:
ordir={"North":0.0,
    "NNE":22.5,
    "NE": 45.0,
    "ENE":67.5,
    "East":90.0,
    "ESE":112.5,
    "SE":135.0,
    "SSE":157.5,
    "South":180,
    "SSW":202.5,
    "SW":225.0,
    "WSW":247.5,
    "West":270.0,
    "WNW":292.5,
    "NW":315.0,
    "NNW":337.5}

In [ ]:
def dailywind(d):
    wdate = d
    url = f"https://www.wunderground.com/dashboard/pws/KCASANTA106/table/{wdate}/{wdate}/daily"
    html = urlopen(url)
    soup = BeautifulSoup(html)

    headers = [th.getText() for th in soup.findAll('thead')[3].findAll('th')]
    rows = soup.findAll('tbody')[3].findAll('tr')
    hourly_data = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]

    df_weather = pd.DataFrame(hourly_data,columns=headers)
    df_wind = df_weather[['Time','Temperature','Wind','Speed','Gust']]
    df_wind['Date'] = wdate
    df_wind['Azimuth']= df_wind['Wind'].map(ordir)

    df_wind = df_wind[['Date','Time','Temperature','Wind','Azimuth','Speed','Gust']]
    df_wind.columns = ['Date', 'Time','Temp','Wind','Azimuth', 'Speed', 'Gust']

    return df_wind

In [ ]:
now = datetime.now().date()
sdate= date(2022,10,5)
datediff = (now-sdate).days
print(f"# full days to pull between start date ({sdate}) and now ({now}): {datediff}")

In [ ]:
nested_dfs=[]
for i in range(datediff):
    weatherdate = sdate + timedelta(days = i)
    nested_dfs.append(dailywind(weatherdate))

In [ ]:
allwind = nested_dfs[0]
for i in range(1,len(nested_dfs)):
    allwind = pd.concat([allwind,nested_dfs[i]],axis=0)
allwind['Speed'] = allwind['Speed'].str[:-5].astype(float)
allwind['Gust'][allwind['Gust']=='--'] = '0.0 °mph'
allwind['Gust'] = allwind['Gust'].str[:-5].astype(float)
allwind['Temp'] = allwind['Temp'].str[:-3].astype(float)
allwind['Month'] = pd.DatetimeIndex(allwind['Date']).month_name().str[:3]

In [ ]:
allwind = pd.concat([,allwind])

In [ ]:
allwind.loc[(allwind.Time.str.startswith(("10:","11:")) & allwind.Time.str.endswith("PM")) |\
            (allwind.Time.str.startswith(("12:","1:","2:","3:","4:","5:")) & allwind.Time.str.endswith("AM")) ,\
            "Daypart"] = "Overnight"

allwind.loc[allwind.Time.str.startswith(("6:","7:","8:","9:")) & \
            allwind.Time.str.endswith("AM"), \
            "Daypart"] = "Morning"

allwind.loc[(allwind.Time.str.startswith(("10:","11:")) & allwind.Time.str.endswith("AM")) |\
            (allwind.Time.str.startswith(("12:","1:")) & allwind.Time.str.endswith("PM")) ,\
            "Daypart"] = "Midday"

allwind.loc[allwind.Time.str.startswith(("2:","3:","4:","5:")) & \
            allwind.Time.str.endswith("PM"), \
            "Daypart"] = "Afternoon"

allwind.loc[allwind.Time.str.startswith(("6:","7:","8:","9:")) & \
            allwind.Time.str.endswith("PM"), \
            "Daypart"] = "Evening"

In [ ]:
allwind.loc[allwind.Month.isin(['Dec','Jan','Feb']), \
            "Season"] = "Winter"

allwind.loc[allwind.Month.isin(['Mar','Apr','May']), \
            "Season"] = "Spring"
             
allwind.loc[allwind.Month.isin(['Jun','Jul','Aug']), \
            "Season"] = "Summer"
             
allwind.loc[allwind.Month.isin(['Sep','Oct','Nov']), \
            "Season"] = "Fall"

In [ ]:
allwind = allwind[['Date','Time','Season','Month','Daypart','Temp','Wind','Azimuth','Speed','Gust']]

In [ ]:
allwind[::len(allwind)//10]

In [ ]:
fday,lday = sorted(allwind.Date)[0],sorted(allwind.Date)[-1]

In [ ]:
allwind.to_pickle(f'brommer_raw_{fday}_{lday}.pkl')
allwind.to_csv(f'brommer_raw_{fday}_{lday}.csv')